# 📊 Completeness Check Notebook
This notebook checks for completeness (missing or present data) in every cell of a CSV file using the column `Sr#` as the unique row identifier.

In [ ]:

# Step 1: Import Libraries
import pandas as pd


In [ ]:

# Step 2: Load the CSV file (same folder as this notebook)
file_path = 'Planview Data.csv'
df = pd.read_csv(file_path)


In [ ]:

# Step 3: Validate 'Sr#' column exists
if 'Sr#' not in df.columns:
    raise ValueError("The column 'Sr#' was not found in the CSV file.")


In [ ]:

# Step 4: Preserve 'Sr#' as row identifier
row_ids = df['Sr#']

# Step 5: Remove 'Sr#' from main data
df_data = df.drop(columns=['Sr#'])

# Step 6: Add original Sr# back as index column for melting
df_data['Index (Row number you created earlier)'] = row_ids


In [ ]:

# Step 7: Melt DataFrame to long format (1 row per cell)
melted_df = df_data.melt(id_vars='Index (Row number you created earlier)',
                         var_name='Column Name',
                         value_name='Value')


In [ ]:

# Step 8: Apply completeness check: 1 = complete, 0 = missing
melted_df['Complete (1 or 0)'] = melted_df['Value'].apply(
    lambda x: 1 if pd.notnull(x) and str(x).strip() != '' else 0
)


In [ ]:

# Step 9: Add sequential Sr# column
melted_df.insert(0, 'Sr#', range(1, len(melted_df) + 1))

# Step 10: Select required columns
final_df = melted_df[['Sr#', 'Column Name', 'Index (Row number you created earlier)', 'Complete (1 or 0)']]


In [ ]:

# Step 11: Split into 4 equal parts and export
num_parts = 4
chunk_size = len(final_df) // num_parts

for i in range(num_parts):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < num_parts - 1 else len(final_df)
    part_df = final_df.iloc[start:end]
    part_df.to_csv(f'completeness_part_{i+1}.csv', index=False)

print("✅ Completed! Exported 4 CSV files for completeness check.")
